In [155]:
import requests # Импортируем библиотеку requests для запросов
from bs4 import BeautifulSoup # Импортируем библиотеку BeautifulSoup для парсинга
import bleach  # Импортируем библиотеку bleach для очистки текста от тэгов
import re      # Импортируем библиотеку для регулярных выражений
import pyautogui  # Импортируем библиотеку для интерактивного ввода
import pandas as pd
import os.path

In [156]:
if os.path.isfile('./all_url_df.csv'):
    # Считываем таблицу ссылок из файла, если он существует.
    url_df = pd.read_csv('./all_url_df.csv')
else:
    # Создаем таблицу таблицу ссылок, если файла с ней нет.
    url_df = pd.DataFrame(columns=['url'])

In [157]:
def grab_page_url(url):
    """
        Функция формирует адрес ссылки на рецепт.
            Args:
                url (string): адрес страницы каталога рецептов.
            Returns:
                url_curr_page_df (dataframe): адрес ссылки на рецепт.
    """
    # Выполняем GET-запрос, содержимое ответа присваивается переменной response
    response = requests.get(url, headers={
    'User-Agent': 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
    # Создаём объект BeautifulSoup, указывая html-парсер
    page = BeautifulSoup(response.text, 'html.parser')
    begin=page.text.find(
        '@type":"ItemList","itemListElement":[{"@type":"ListItem","url"')
    end=page.text.find('"@context":"http://schema.org"')
    page.text[begin:end]
    url_list_r = page.text[begin:end].split('{"@type":"ListItem","url":"')
    url_list = []
    for i in range(1, len(url_list_r)):
        url_list.append(url_list_r[i].split(',')[0][:-1])

    url_curr_page_df = pd.DataFrame(columns=['url'])
    url_curr_page_df.url = pd.Series(url_list)
    return url_curr_page_df

In [158]:
for i in range(100,714):
    url = 'https://eda.ru/recepty?page='+str(i)
    # Присоединение url текущей страницы  к таблице url
    url_df = pd.concat([url_df, grab_page_url(url)], ignore_index=True)
    # удаление дубликатов рецептов 
    url_df.drop_duplicates(inplace=True)

# Запись в файл
url_df.to_csv('./all_urls_df.csv', index=False)
    